##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 第2天 - 使用 Chroma 实现基于 RAG 的文档问答系统

欢迎回到 Kaggle 5天生成式 AI 课程!

**注意**: 第1天的笔记本包含了很多关于如何设置 Kaggle Notebooks 的信息。如果你遇到任何问题, 请[查看那里的故障排除步骤](https://www.kaggle.com/code/markishere/day-1-prompting#Get-started-with-Kaggle-notebooks)。

LLMs 有两个主要限制: 1) 它们只"知道"训练数据中的信息, 2) 它们的输入上下文窗口有限。解决这两个限制的一个方法是使用一种叫做检索增强生成(Retrieval Augmented Generation, RAG)的技术。RAG 系统包含三个阶段:

1. 索引
2. 检索
3. 生成

索引是预先进行的, 它允许你在查询时快速查找相关信息。当收到查询时, 你检索相关文档, 将它们与你的指令和用户的查询结合起来, 然后让 LLM 使用提供的信息生成定制的自然语言答案。这使你能够提供模型之前没有见过的信息, 比如特定产品的知识或实时天气更新。

在这个笔记本中, 你将使用 Gemini API 创建一个向量数据库, 从数据库中检索问题的答案并生成最终答案。你将使用 [Chroma](https://docs.trychroma.com/), 这是一个开源的向量数据库。使用 Chroma, 你可以将嵌入向量与元数据一起存储, 嵌入文档和查询, 并搜索你的文档。

## 获取帮助

**常见问题都已在[FAQ 和故障排除指南](https://www.kaggle.com/code/markishere/day-0-troubleshooting-and-faqs)中covered。**

## Setup

First, install ChromaDB and the Gemini API Python SDK.

In [ ]:
# %pip install -U -q "google-generativeai>=0.8.3" chromadb
%pip install -U -q chromadb

In [2]:
import google.generativeai as genai
from IPython.display import Markdown

### Set up your API key

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [3]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

If you received an error response along the lines of `No user secrets exist for kernel id ...`, then you need to add your API key via `Add-ons`, `Secrets` **and** enable it.

![Screenshot of the checkbox to enable GOOGLE_API_KEY secret](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_3.png)

### 探索可用的模型
 
在本指南中，您将使用 [`embedContent`](https://ai.google.dev/api/embeddings#method:-models.embedcontent) API 方法来计算嵌入。通过 [`models.list`](https://ai.google.dev/api/models#method:-models.list) 端点找到支持它的模型。您还可以在 [模型页面](https://ai.google.dev/gemini-api/docs/models/gemini#text-embedding) 上找到有关嵌入模型的更多信息。
 
`text-embedding-004` 是最新的嵌入模型，因此您将在本练习中使用它。

In [ ]:
for m in genai.list_models():
    if "embedContent" in m.supported_generation_methods:
        print(m.name)

### 数据

这里有一小组文档，您将使用它们来创建嵌入数据库。

In [5]:
DOCUMENT1 = "操作气候控制系统  您的Googlecar配备了一个气候控制系统，可以让您调节车内的温度和气流。要操作气候控制系统，请使用位于中央控制台上的按钮和旋钮。 温度：温度旋钮控制车内的温度。顺时针旋转旋钮以增加温度，逆时针旋转以降低温度。气流：气流旋钮控制车内的气流量。顺时针旋转旋钮以增加气流，逆时针旋转以减少气流。风扇速度：风扇速度旋钮控制风扇的速度。顺时针旋转旋钮以增加风扇速度，逆时针旋转以降低风扇速度。模式：模式按钮允许您选择所需的模式。可用的模式有：自动：汽车将自动调节温度和气流以保持舒适水平。冷却：汽车将吹入冷空气。加热：汽车将吹入暖空气。除霜：汽车将暖空气吹到挡风玻璃上以除霜。"
DOCUMENT2 = '您的Googlecar配备了一个大型触摸屏显示器，可以访问各种功能，包括导航、娱乐和气候控制。要使用触摸屏显示器，只需触摸所需的图标。例如，您可以触摸“导航”图标以获取到达目的地的路线，或触摸“音乐”图标以播放您喜欢的歌曲。'
DOCUMENT3 = "换挡 您的Googlecar配备了自动变速器。要换挡，只需将换挡杆移动到所需位置。 停车：此位置用于停车时。车轮被锁定，汽车无法移动。倒车：此位置用于倒车。空档：此位置用于在红灯或交通中停车时。汽车不在档位，除非您踩油门，否则不会移动。前进：此位置用于向前行驶。低速：此位置用于在雪地或其他滑溜的条件下行驶。"

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

## 使用 ChromaDB 创建嵌入数据库

创建一个[自定义函数](https://docs.trychroma.com/guides/embeddings#custom-embedding-functions)来使用 Gemini API 生成嵌入向量。在这个任务中, 你正在实现一个检索系统, 所以生成*文档*嵌入的 `task_type` 是 `retrieval_document`。稍后, 你将使用 `retrieval_query` 来生成*查询*嵌入。查看 [API 参考文档](https://ai.google.dev/api/embeddings#v1beta.TaskType)获取所有支持的任务类型列表。

In [ ]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry

# 定义一个自定义的嵌入函数类，用于生成文档或查询的嵌入
class GeminiEmbeddingFunction(EmbeddingFunction):
    # 指定是为文档生成嵌入还是为查询生成嵌入
    document_mode = True

    # 定义调用方法，输入为文档，输出为嵌入向量
    def __call__(self, input: Documents) -> Embeddings:
        # 根据模式选择嵌入任务类型
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        # 定义重试策略，处理临时错误
        retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}

        # 调用 Gemini API 的 embed_content 方法生成嵌入
        response = genai.embed_content(
            model="models/text-embedding-004",  # 使用的嵌入模型
            content=input,  # 输入内容
            task_type=embedding_task,  # 任务类型
            request_options=retry_policy,  # 请求选项，包括重试策略
        )
        # 返回嵌入向量
        return response["embedding"]

现在创建一个 [Chroma database client](https://docs.trychroma.com/getting-started)，它使用 `GeminiEmbeddingFunction` 并用你上面定义的文档填充数据库。

In [ ]:
# 导入 ChromaDB 库
import chromadb

# 设置数据库名称
DB_NAME = "googlecardb"

# 创建 Gemini 嵌入函数实例
embed_fn = GeminiEmbeddingFunction()
# 设置为文档嵌入模式
embed_fn.document_mode = True

# 创建 ChromaDB 客户端
chroma_client = chromadb.Client()
# 获取或创建集合，并指定使用的嵌入函数
db = chroma_client.get_or_create_collection(
    name=DB_NAME, 
    embedding_function=embed_fn
)

# 将文档添加到数据库中
# 为每个文档生成一个字符串ID (从"0"开始)
db.add(
    documents=documents,
    ids=[str(i) for i in range(len(documents))]
)

Confirm that the data was inserted by looking at the database.

In [ ]:
db.count()
# You can peek at the data too.
db.peek(1)

## 检索：查找相关文档
 
要搜索 Chroma 数据库，请调用 `query` 方法。请注意，您还需要切换到嵌入生成的 `retrieval_query` 模式。


In [ ]:
# 切换到查询模式生成嵌入向量
embed_fn.document_mode = False

# 定义要搜索的问题
query = "如果我在车上想听音乐，我该怎么做？"

# 在 Chroma 数据库中搜索相关文档
# n_results=1 表示只返回最相关的一个结果
result = db.query(
    query_texts=[query], 
    n_results=1
)

# 从结果中解构出文档内容
# result["documents"] 返回嵌套列表 [[passage]]，使用列表解构获取 passage
[[passage]] = result["documents"]

# 使用 Markdown 格式显示检索到的文档
Markdown(passage)

## 增强生成：回答问题

现在你已经从文档集中找到了一个相关段落（*检索*步骤），你现在可以组装一个生成提示，让 Gemini API *生成*一个最终答案。请注意，在这个例子中只检索到一个段落。实际上，特别是当你的底层数据量很大时，你会希望检索多个结果，并让 Gemini 模型确定哪些段落与回答问题相关。因此，如果某些检索到的段落与问题没有直接关系，这个生成步骤应该忽略它们。

In [ ]:
passage_oneline = passage.replace("\n", " ")
query_oneline = query.replace("\n", " ")

# This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
prompt = f"""你是一个乐于助人且信息丰富的机器人，使用下面包含的参考段落中的文本回答问题。
请务必用完整的句子回答，全面且包含所有相关的背景信息。
但是，你的听众是非技术人员，所以一定要分解复杂的概念，并保持友好和对话的语气。如果段落与答案无关，你可以忽略它。

QUESTION: {query_oneline}
PASSAGE: {passage_oneline}
"""
print(prompt)

Now use the `generate_content` method to to generate an answer to the question.

In [ ]:
# 创建一个生成模型实例，使用最新的 gemini-1.5-flash 版本
model = genai.GenerativeModel("gemini-1.5-flash-latest")

# 使用生成模型生成内容，传入之前构建的提示
answer = model.generate_content(prompt)

# 使用 Markdown 格式显示生成的答案
Markdown(answer.text)

## 下一步

恭喜你构建了一个检索增强生成应用！

要了解更多关于在 Gemini API 中使用嵌入的信息，请查看 [嵌入简介](https://ai.google.dev/gemini-api/docs/embeddings) 或者学习机器学习速成课程的 [嵌入章节](https://developers.google.com/machine-learning/crash-course/embeddings) 以了解更多基础知识。

对于托管的 RAG 系统，请查看 Gemini API 中的 [语义检索服务](https://ai.google.dev/gemini-api/docs/semantic_retrieval)。你可以在单个请求中实现对自己文档的问答，或者托管一个数据库以获得更快的响应。